In [1]:
import re
import os
from misc import *
import numpy as np
import pandas as pd
import pickle as pkl
import os.path as op
from tqdm import tqdm
from copy import deepcopy
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from scipy.stats import ks_2samp, median_test
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.stats import spearmanr, pearsonr, ks_2samp, chisquare, levene

In [2]:
SHAM = "../data/sham_no stimulation/"
vlPFC = "../data/vlPFC_stimulation/"
enc_on = "../data/enc_on_new/"
enc_off = "../data/enc_off_new/"
eng_sham = "../data/Eng_sham/"
eng_vlPFC = "../data/Eng_vlPFC/"

In [3]:
get_files = lambda x: [op.join(x, b) for b in [a for a in os.walk(x)][0][2]]

In [4]:
sham_study = sum([Study.load_from_file(a) for a in get_files(SHAM)])
vlPFC_study = sum([Study.load_from_file(a) for a in get_files(vlPFC)])
eon_study = sum([Study.load_from_file(a) for a in get_files(enc_on)])
eoff_study = sum([Study.load_from_file(a) for a in get_files(enc_off)])
esham_study = sum([Study.load_from_file(a) for a in get_files(eng_sham)])
evlPFC_study = sum([Study.load_from_file(a) for a in get_files(eng_vlPFC)])

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'1,214,??'

In [ ]:
len(sham_study), len(vlPFC_study)

In [ ]:
sham_exclude = [
    "tDCS3_101_test.xlsx", "tDCS3_103_test.xlsx", "tDCS3_132_test.xlsx", 
    "tDCS3_198_test.xlsx", "tDCS3_209_test.xlsx", "tDCS3_221_test.xlsx"
]
vl_exclude = ["tDCS3_170_test.xlsx", "tDCS3_207_test.xlsx"]
sham_exclude = sum([Study.load_from_file(op.join(SHAM, a)) for a in sham_exclude])
vl_exclude = sum([Study.load_from_file(op.join(vlPFC, a)) for a in vl_exclude])

In [ ]:
sham_study = sham_study-sham_exclude
vlPFC_study = vlPFC_study-vl_exclude

In [ ]:
len(sham_study), len(vlPFC_study)

In [ ]:
fnames = [
    "sham_study.pkl", "vlPFC_study.pkl", "eon_study.pkl", "eoff_study.pkl", 
    "esham_study.pkl", "evlPFC_study.pkl"
]
for a,b in zip([sham_study, vlPFC_study, eon_study, eoff_study, esham_study, evlPFC_study], fnames):
    a.save(b)

### Figure 1

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 8, 3

In [ ]:
plt.subplot(1,2,1)
plt.title("A")
plt.violinplot(
    [sham_study.compute_study_aucs(), vlPFC_study.compute_study_aucs()]#, eon_auc, eoff_auc]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Sham", "VLPFC online"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.ylabel("AUROC")
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.subplot(1,2,2)
plt.violinplot(
    [esham_study.compute_study_aucs(), evlPFC_study.compute_study_aucs()]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Sham", "VLPFC Online"])
#plt.grid(True)
plt.ylim(0,1)
plt.title("B")
#plt.ylabel("AUC")
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.show()

In [ ]:
plt.subplot(1,2,1)
plt.boxplot(
    [sham_study.compute_study_aucs(), vlPFC_study.compute_study_aucs()], notch=True#, eon_auc, eoff_auc]
)
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Sham", "VLPFC online"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.ylabel("AUC")
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.subplot(1,2,2)
plt.boxplot(
    [esham_study.compute_study_aucs(), evlPFC_study.compute_study_aucs()], notch=True
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Sham", "VLPFC Online"])
#plt.grid(True)
plt.ylim(0,1)
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.ylabel("AUC")
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.show()

In [ ]:
median_test(sham_study.compute_study_aucs(), vlPFC_study.compute_study_aucs())

### Figure 2

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

In [ ]:
ru_study = sham_study+vlPFC_study#+eoff_study+eon_study
en_study = esham_study+evlPFC_study

In [ ]:
ru_study_all = ru_study+eoff_study+eon_study

In [ ]:
ru_study_all.save("ru_study.pkl")
en_study.save("en_study.pkl")

In [ ]:
ru_word_aucs = ru_study.compute_word_aucs()
en_word_aucs = en_study.compute_word_aucs()
rusham_word_aucs = sham_study.compute_word_aucs()
esham_word_aucs = esham_study.compute_word_aucs()
ruvlPFC_word_aucs = vlPFC_study.compute_word_aucs()
evlPFC_word_aucs = evlPFC_study.compute_word_aucs()

In [ ]:
plt.subplot(1,3,1)
plt.title("A")
plt.violinplot(
    [ru_word_aucs[0], en_word_aucs[0]]#, eon_auc, eoff_auc]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Russian", "English"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.ylabel("AUROC")
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.subplot(1,3,2)
plt.title("B")
plt.violinplot(
    [rusham_word_aucs[0], esham_word_aucs[0]]#, eon_auc, eoff_auc]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Russian", "English"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.subplot(1,3,3)
plt.title("C")
plt.violinplot(
    [ruvlPFC_word_aucs[0], evlPFC_word_aucs[0]]#, eon_auc, eoff_auc]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Russian", "English"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.show()

In [ ]:
plt.subplot(1,3,1)
plt.title("A")
plt.boxplot(
    [ru_word_aucs[0], en_word_aucs[0]], notch=True#, eon_auc, eoff_auc]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Russian", "English"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.ylabel("AUROC")
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.subplot(1,3,2)
plt.title("B")
plt.boxplot(
    [rusham_word_aucs[0], esham_word_aucs[0]], notch=True#, eon_auc, eoff_auc]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Russian", "English"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.subplot(1,3,3)
plt.title("C")
plt.boxplot(
    [ruvlPFC_word_aucs[0], evlPFC_word_aucs[0]], notch=True#, eon_auc, eoff_auc]
)
plt.xlim(0.5, 2.5)
plt.xticks([1,2], ["Russian", "English"])#, "DLPFC online", "DLPFC offline"])
#plt.grid(True)
plt.ylim(0,1)
plt.axhline(y=0.5, xmin=0, xmax=3, color="red")
plt.legend(
    loc="best",
    handles=[Patch(color="red", label="AUROC=0.5")]
)
plt.show()

In [ ]:
median_test(ruvlPFC_word_aucs[0], evlPFC_word_aucs[0])

### Figure 3

In [ ]:
ru_word_RTs = ru_study.get_word_RT()
en_word_RTs = en_study.get_word_RT()

In [ ]:
ru_RTs = np.array(sum([ru_word_RTs[a] for a in ru_word_RTs], []))
en_RTs = np.array(sum([en_word_RTs[a] for a in en_word_RTs], []))

In [ ]:
ru_RTs = ru_study.get_all_RT()
en_RTs = en_study.get_all_RT()

In [ ]:
plt.hist(Study.clamp(ru_RTs, 2), bins=20, color=(1,0,0,0.5))
plt.hist(Study.clamp(en_RTs, 2), bins=20, color=(0,0,1,0.5))
plt.legend(
    loc="best",
    handles=[
        Patch(color=(1,0,0,0.5), label="Russian"),
        Patch(color=(0,0,1,0.5), label="English"),
    ]
)
plt.xlabel("Reaction time")
plt.ylabel("#words")
plt.show()

In [ ]:
ks_2samp(
    Study.clamp(ru_RTs, 2), Study.clamp(en_RTs, 2)
)

In [ ]:
median_test(
    Study.clamp(ru_RTs, 2), Study.clamp(en_RTs, 2)
)

In [ ]:
median_test(
    ru_RTs, en_RTs
)